### ============================================================
## SILVER → GOLD TRANSFORMATION PIPELINE
### ============================================================
### Purpose:
###   - Read Silver Delta table
###   - Apply business logic, KPIs, aggregations
###   - Produce Gold Delta table (business-ready)
### ============================================================


In [0]:
#Authorization
spark.conf.set(
  "fs.azure.account.key.<container>.blob.core.windows.net",
  "<security_key>"
)

In [0]:
#READ SILVER TABLE
df_silver = spark.table("<custom>.default.project2_silver")
df_silver.createOrReplaceTempView("silver")

In [0]:
#GOLD TRANSFORMATION (BUSINESS LOGIC)
df_gold = spark.sql("""
    WITH base AS (
        SELECT
            device_id,
            timestamp,
            temperature,
            humidity,
            status,
            DATE(timestamp) AS event_date
        FROM silver
    ),

    rolling AS (
        SELECT
            *,
            AVG(temperature) OVER (
                PARTITION BY device_id
                ORDER BY timestamp
                ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
            ) AS temp_rolling_avg,
            AVG(humidity) OVER (
                PARTITION BY device_id
                ORDER BY timestamp
                ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
            ) AS hum_rolling_avg
        FROM base
    ),

    error_streaks AS (
        SELECT
            *,
            SUM(CASE WHEN status = 'error' THEN 1 ELSE 0 END)
                OVER (
                    PARTITION BY device_id
                    ORDER BY timestamp
                    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                ) AS cumulative_errors
        FROM rolling
    ),

    daily_summary AS (
        SELECT
            device_id,
            event_date,
            COUNT(*) AS total_records,

            -- status counts
            SUM(CASE WHEN status = 'ok' THEN 1 ELSE 0 END) AS ok_count,
            SUM(CASE WHEN status = 'warning' THEN 1 ELSE 0 END) AS warning_count,
            SUM(CASE WHEN status = 'error' THEN 1 ELSE 0 END) AS error_count,

            -- averages
            AVG(temperature) AS avg_temperature,
            AVG(humidity) AS avg_humidity,
            AVG(temp_rolling_avg) AS avg_temp_rolling,
            AVG(hum_rolling_avg) AS avg_hum_rolling,

            -- cumulative errors
            MAX(cumulative_errors) AS total_errors_to_date
        FROM error_streaks
        GROUP BY device_id, event_date
    )

    SELECT * FROM daily_summary
""")



In [0]:
#WRITE GOLD TABLE (UC‑MANAGED DELTA)
df_gold.write.format("delta").mode("overwrite").saveAsTable("<custom>.default.project2_gold")


In [0]:
%sql
-- ============================================================
-- VALIDATION / INSPECTION
-- Validation only. Not part of pipeline logic.
-- ============================================================

SELECT * FROM <custom>.default.project2_gold ORDER BY device_id, event_date LIMIT 50;

device_id,event_date,total_records,error_count,avg_temperature,avg_humidity,avg_temp_rolling,avg_hum_rolling,total_errors_to_date,ok_count,warning_count
dev-01,2026-01-01,67,16,-35.918358333075226,49.41044778966192,-35.81895534553338,49.809527449346895,16,0,0
dev-02,2026-01-01,64,20,-7.281249895691872,50.811718851327896,-7.227343638738006,50.67174485325812,20,0,0
dev-03,2026-01-01,55,19,5.478000016645952,48.49400017478249,5.469242448517773,47.84030325918487,19,0,0


In [0]:
# ============================================
# PIPELINE VALIDATION (FAIL IF NO DATA)
# ============================================

row_count = spark.sql("""
    SELECT COUNT(*) AS row_count
    FROM uzi.default.project2_gold
""").first()["row_count"]

if row_count == 0:
    raise Exception("Gold table validation failed: row_count = 0")


In [0]:
#Log the count
print(f"Gold table row_count = {row_count}")

Gold table row_count = 3


In [0]:
%sql
--VISUALIZATIONS
SELECT
    device_id,
    event_date,

    -- status counts (your new Gold logic)
    ok_count,
    warning_count,
    error_count,

    -- totals and averages
    total_records,
    avg_temperature,
    avg_humidity,
    avg_temp_rolling,
    avg_hum_rolling,
    total_errors_to_date,

    -- visualization helpers
    total_records AS events_per_device_day,

    SUM(total_records) OVER (
        ORDER BY event_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_events

FROM <custom>.default.project2_gold
ORDER BY device_id, event_date;


device_id,event_date,ok_count,warning_count,error_count,total_records,avg_temperature,avg_humidity,avg_temp_rolling,avg_hum_rolling,total_errors_to_date,events_per_device_day,cumulative_events
dev-01,2026-01-01,0,0,16,67,-35.918358333075226,49.41044778966192,-35.81895534553338,49.809527449346895,16,67,67
dev-02,2026-01-01,0,0,20,64,-7.281249895691872,50.811718851327896,-7.227343638738006,50.67174485325812,20,64,131
dev-03,2026-01-01,0,0,19,55,5.478000016645952,48.49400017478249,5.469242448517773,47.84030325918487,19,55,186


Databricks visualization. Run in Databricks to view.